# Cifar-10

## 1.Gaussian weights

In [3]:
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torch.nn.functional as F
import torchvision.transforms as transforms

In [4]:
# 定义训练集变换
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomCrop(32, padding=4),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 载入训练集和测试集
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)

# 定义训练集和测试集的dataloader
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)  
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

100%|██████████| 170498071/170498071 [00:05<00:00, 28526600.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [5]:
labels = []
for x, y in train_loader:
    labels.extend(y.tolist())
labels = set(labels)
n_classes = len(labels) + 1  
print(n_classes)  

11


### 1.1 VI

In [6]:
# Model with Gaussian prior 
class BNN(nn.Module):
      def __init__(self, n_in, n_out,n_hide):
        super().__init__()
        self.w = nn.Parameter(torch.randn(n_in, n_out))
        self.fc1 = nn.Linear(n_in, n_hide) 
        self.fc2 = nn.Linear(n_hide, n_out)
        self.dropout = nn.Dropout(0.5) 
    
      def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  
  
      def kl_divergence(self): 
        mean, std = self.w.mean(), self.w.std()
        return ((mean ** 2 + std ** 2 - torch.log(std **2)-1)/2).sum()

In [7]:
model = BNN(3 * 32 * 32, 10, 512)

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(50):
    for x, y in train_loader:
        x = x.view(-1, 32*32*3)
        loss = F.nll_loss(model(x), y)
        loss += model.kl_divergence() * 0.1   #添加KL散度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Train Loss: {loss.item():.3f}')

Epoch: 1, Train Loss: 2.071
Epoch: 2, Train Loss: 1.897
Epoch: 3, Train Loss: 1.974
Epoch: 4, Train Loss: 1.933
Epoch: 5, Train Loss: 1.916
Epoch: 6, Train Loss: 1.816
Epoch: 7, Train Loss: 1.668
Epoch: 8, Train Loss: 1.868
Epoch: 9, Train Loss: 2.018
Epoch: 10, Train Loss: 1.717
Epoch: 11, Train Loss: 1.739
Epoch: 12, Train Loss: 1.626
Epoch: 13, Train Loss: 1.727
Epoch: 14, Train Loss: 1.748
Epoch: 15, Train Loss: 1.763
Epoch: 16, Train Loss: 1.604
Epoch: 17, Train Loss: 1.573
Epoch: 18, Train Loss: 1.769
Epoch: 19, Train Loss: 1.688
Epoch: 20, Train Loss: 1.789
Epoch: 21, Train Loss: 1.753
Epoch: 22, Train Loss: 1.758
Epoch: 23, Train Loss: 1.700
Epoch: 24, Train Loss: 1.786
Epoch: 25, Train Loss: 1.572
Epoch: 26, Train Loss: 1.704
Epoch: 27, Train Loss: 1.697
Epoch: 28, Train Loss: 1.646
Epoch: 29, Train Loss: 1.757
Epoch: 30, Train Loss: 1.503
Epoch: 31, Train Loss: 1.625
Epoch: 32, Train Loss: 1.706
Epoch: 33, Train Loss: 1.634
Epoch: 34, Train Loss: 1.697
Epoch: 35, Train Loss: 

In [9]:
# Test accuracy
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        x = x.view(-1, 32*32*3)
        y_pred = model(x).argmax(dim=1)
        correct += (y_pred == y).sum().item()

print(f'Test accuracy: {correct/len(test_dataset):.3f}')

Test accuracy: 0.436


### 1.2MCMC

In [10]:
# Get train data
train_x = []  
train_y = []
for x, y in train_loader:
    train_x.append(x.view(-1, 32*32*3))
    train_y.append(y)
train_x = torch.cat(train_x)  
train_y = torch.cat(train_y)

In [11]:
# Model with Gaussian prior 
class BNN(nn.Module):
    def __init__(self, n_in, n_hide, n_out):
        super().__init__()
        self.fc1 = nn.Linear(n_in, n_hide)  
        self.fc2 = nn.Linear(n_hide, n_out)
        self.dropout = nn.Dropout(0.5)
        self.w = torch.zeros(n_in, n_out)  # Weight parameter
        
    def forward(self, x,w_proposal=None):
        if w_proposal is None:
            w = self.w
        else:
            w = w_proposal
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        pred = F.log_softmax(x, dim=1)
        dist = torch.distributions.Categorical(logits=pred)
        return dist

In [12]:
# MCMC Sampling
model = BNN(3 * 32 * 32, 512, 10) 
w = model.w.data  # Get weight tensor 

In [13]:
for i in range(1500):  # 1500 iterations
    # Proposal distribution
    w_proposal = w + torch.randn(w.size())*0.1 

    # Acceptance ratio
    ap = torch.exp(model(train_x).log_prob(train_y).sum() - model(train_x, w_proposal).log_prob(train_y).sum()) 

    # Metropolis acceptance
    u = torch.rand(1)
    if u < ap:
        w = w_proposal  # Accept proposal 

    model.w.data = w  # Set weight to sampled value

In [14]:
# Test accuracy
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        x = x.view(-1, 32*32*3)
        y_pred = model(x).sample()
        correct += (y_pred == y).sum().item()
    print(f'Test accuracy: {correct/len(test_dataset):.3f}') 

Test accuracy: 0.100


## 2.Laplace weights

### 2.1Variable Inference

In [15]:
# Model with Laplace prior 
class BNN(nn.Module):
    def __init__(self, n_in, n_out,n_hide):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(n_in, n_out))
        self.fc1 = nn.Linear(n_in, n_hide)  
        self.fc2 = nn.Linear(n_hide, n_out)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  

    def kl_divergence(self):
        return (self.w.abs().sum() / 2).sum()  # Laplace prior  

In [16]:
model = BNN(3 * 32 * 32, 10, 512) 

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(50):
    for x, y in train_loader:
        x = x.view(-1, 32*32*3)
        loss = F.nll_loss(model(x), y)
        loss += model.kl_divergence() * 0.1   
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Train Loss: {loss.item():.3f}')


Epoch: 1, Train Loss: 2.041
Epoch: 2, Train Loss: 1.893
Epoch: 3, Train Loss: 1.946
Epoch: 4, Train Loss: 1.901
Epoch: 5, Train Loss: 1.772
Epoch: 6, Train Loss: 1.841
Epoch: 7, Train Loss: 1.635
Epoch: 8, Train Loss: 1.758
Epoch: 9, Train Loss: 1.729
Epoch: 10, Train Loss: 1.812
Epoch: 11, Train Loss: 1.634
Epoch: 12, Train Loss: 1.668
Epoch: 13, Train Loss: 1.778
Epoch: 14, Train Loss: 1.807
Epoch: 15, Train Loss: 1.705
Epoch: 16, Train Loss: 1.832
Epoch: 17, Train Loss: 1.796
Epoch: 18, Train Loss: 1.548
Epoch: 19, Train Loss: 1.737
Epoch: 20, Train Loss: 1.775
Epoch: 21, Train Loss: 1.777
Epoch: 22, Train Loss: 1.768
Epoch: 23, Train Loss: 1.751
Epoch: 24, Train Loss: 1.671
Epoch: 25, Train Loss: 1.705
Epoch: 26, Train Loss: 1.671
Epoch: 27, Train Loss: 1.640
Epoch: 28, Train Loss: 1.730
Epoch: 29, Train Loss: 1.633
Epoch: 30, Train Loss: 1.544
Epoch: 31, Train Loss: 1.575
Epoch: 32, Train Loss: 1.645
Epoch: 33, Train Loss: 1.672
Epoch: 34, Train Loss: 1.640
Epoch: 35, Train Loss: 

In [18]:
# Test accuracy  
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        x = x.view(-1, 32*32*3)
        y_pred = model(x).argmax(dim=1)
        correct += (y_pred == y).sum().item() 
print(f'Test accuracy: {correct/len(test_dataset):.3f}') 

Test accuracy: 0.432


### 2.2MCMC

In [19]:
# Get train data
train_x = []  
train_y = []
for x, y in train_loader:
    train_x.append(x.view(-1, 32*32*3))
    train_y.append(y)
train_x = torch.cat(train_x)  
train_y = torch.cat(train_y)

In [20]:
# Model with Laplace prior
class BNN(nn.Module):
    def __init__(self, n_in, n_hide, n_out):
        super().__init__()
        self.fc1 = nn.Linear(n_in, n_hide)  
        self.fc2 = nn.Linear(n_hide, n_out)
        self.dropout = nn.Dropout(0.5)
        self.w = torch.zeros(n_in, n_out)  

    def forward(self, x, w_proposal=None):
        if w_proposal is None:  
              w = self.w
        else:
              w = w_proposal
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        pred = F.log_softmax(x, dim=1)
        dist = torch.distributions.Categorical(logits=pred)
        return dist

    def log_prior(self, w):
        return -w.abs().sum() 

In [21]:
# MCMC Sampling
model = BNN(3 * 32 * 32, 512, 10) 
w = model.w.data  # Get weight tensor 

In [22]:
# training
for i in range(1500): 
    # Proposal distribution
    w_proposal = w + torch.randn(w.size())*0.1

    # Prior ratio 
    lp = model.log_prior(w_proposal)  
    ap = torch.exp(lp - model.log_prior(w))

    # Metropolis acceptance 
    u = torch.rand(1)
    if u < ap:
        w = w_proposal  

    model.w.data = w 

In [23]:
# Test accuracy
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        x = x.view(-1, 32*32*3)
        y_pred = model(x).sample()
        correct += (y_pred == y).sum().item()
    print(f'Test accuracy: {correct/len(test_dataset):.3f}') 

Test accuracy: 0.094


## 3.Equalisation weights

### 3.1Variable Inference

In [24]:
# Model with Uniform prior
class BNN(nn.Module):
    def __init__(self, n_in, n_out,n_hide):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(n_in, n_out))
        self.fc1 = nn.Linear(n_in, n_hide)
        self.fc2 = nn.Linear(n_hide, n_out)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  

    def kl_divergence(self):
        return 0.5*(self.w**2).sum()  # Uniform prior  

In [25]:
model = BNN(3 * 32 * 32, 10, 512) 

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(50):
    for x, y in train_loader:
        x = x.view(-1, 32*32*3)
        loss = F.nll_loss(model(x), y)
        loss += model.kl_divergence() * 0.1   
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Train Loss: {loss.item():.3f}')


Epoch: 1, Train Loss: 2.039
Epoch: 2, Train Loss: 1.843
Epoch: 3, Train Loss: 1.890
Epoch: 4, Train Loss: 1.917
Epoch: 5, Train Loss: 1.791
Epoch: 6, Train Loss: 1.765
Epoch: 7, Train Loss: 1.781
Epoch: 8, Train Loss: 1.840
Epoch: 9, Train Loss: 1.627
Epoch: 10, Train Loss: 1.806
Epoch: 11, Train Loss: 1.772
Epoch: 12, Train Loss: 1.738
Epoch: 13, Train Loss: 1.781
Epoch: 14, Train Loss: 1.663
Epoch: 15, Train Loss: 1.758
Epoch: 16, Train Loss: 1.823
Epoch: 17, Train Loss: 1.709
Epoch: 18, Train Loss: 1.732
Epoch: 19, Train Loss: 1.607
Epoch: 20, Train Loss: 1.673
Epoch: 21, Train Loss: 1.741
Epoch: 22, Train Loss: 1.651
Epoch: 23, Train Loss: 1.696
Epoch: 24, Train Loss: 1.758
Epoch: 25, Train Loss: 1.649
Epoch: 26, Train Loss: 1.755
Epoch: 27, Train Loss: 1.613
Epoch: 28, Train Loss: 1.595
Epoch: 29, Train Loss: 1.673
Epoch: 30, Train Loss: 1.652
Epoch: 31, Train Loss: 1.691
Epoch: 32, Train Loss: 1.454
Epoch: 33, Train Loss: 1.593
Epoch: 34, Train Loss: 1.654
Epoch: 35, Train Loss: 

In [27]:
# Test accuracy  
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        x = x.view(-1, 32*32*3)
        y_pred = model(x).argmax(dim=1)
        correct += (y_pred == y).sum().item() 
print(f'Test accuracy: {correct/len(test_dataset):.3f}') 

Test accuracy: 0.423


### 3.2MCMC

In [28]:
# Get train data
train_x = []  
train_y = []
for x, y in train_loader:
    train_x.append(x.view(-1, 32*32*3))
    train_y.append(y)
train_x = torch.cat(train_x)  
train_y = torch.cat(train_y)

In [29]:
# Model with Uniform prior
class BNN(nn.Module):
    def __init__(self, n_in, n_hide, n_out):
        super().__init__()
        self.fc1 = nn.Linear(n_in, n_hide)  
        self.fc2 = nn.Linear(n_hide, n_out)
        self.dropout = nn.Dropout(0.5)
        self.w = torch.zeros(n_in, n_out)  # Weight parameter

    def forward(self, x, w_proposal=None):
        if w_proposal is None:
              w = self.w
        else:
              w = w_proposal
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        pred = F.log_softmax(x, dim=1)
        dist = torch.distributions.Categorical(logits=pred)
        return dist

    def log_prior(self, w):
        return -torch.sum(torch.abs(w))

In [30]:
# MCMC Sampling
model = BNN(3 * 32 * 32, 512, 10) 
w = model.w.data  # Get weight tensor 

In [31]:
for i in range(1500): 
    w_proposal = torch.rand(w.size())

    ap = torch.tensor(0.)  
    
    u = torch.rand(1)
    if u < ap:  
          w = w_proposal
            
    # update
    model.w.data = w  

    # loss
    logits = model(train_x, w_proposal).logits
    loss = F.cross_entropy(logits, train_y)  

In [32]:
# Test accuracy
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        x = x.view(-1, 32*32*3)
        y_pred = model(x).sample()
        correct += (y_pred == y).sum().item()
    print(f'Test accuracy: {correct/len(test_dataset):.3f}') 

Test accuracy: 0.104
